# Install

In [1]:
!pip install yandex-cloud-ml-sdk
!pip install gigachat langchain-gigachat
!pip install groq

# Импорт

In [ ]:
#------Yandex GPT------
from yandex_cloud_ml_sdk import YCloudML
from yandex_cloud_ml_sdk.auth import APIKeyAuth

#------Giga Chat------
from gigachat import GigaChat
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_gigachat.chat_models import GigaChat
from langchain_core.output_parsers import StrOutputParser

#------Groq------
from groq import Groq

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time

from sklearn.metrics import accuracy_score

# Загрузка статей

In [ ]:
path = '/content/drive/MyDrive/Курсовые/Transformers 2/'
path = '/content/drive/MyDrive/Курсовая/'

df = pd.read_excel(path + "Рефераты_моделей.xlsx")
df.head(2)

,text,summary,rubert_ext_sum,rut5_abs_sum,gpt_abs_sum,YandexGPT5_PRO_RC,GigaChat_2_Lite,meta-llama/llama-4-scout-17b-16e-instruct,deepseek-r1-distill-llama-70b,ministral-8b-latest,mistral-large-latest
0,Американские вооруженные силы должны быть гото...,Экс-президент США продолжает критиковать админ...,Американские вооруженные силы должны быть гото...,"Бывший глава Белого дома Дональд Трамп заявил,...","Бывший президент США Дональд Трамп заявил, что...",NaN,NaN,"Дональд Трамп заявил, что американские вооруже...","Дональд Трамп заявил, что США должны быть гото...","Экс-президент США Дональд Трамп заявил, что ам...","Бывший президент США Дональд Трамп заявил, что..."
1,"Международный коллектив ученых из Гарварда , П...",Международный коллектив под руководством гарва...,"Международный коллектив ученых из Гарварда , П...","Международный коллектив ученых из Гарварда, Пр...",Ученые из Гарварда и Принстона объявили о нача...,Международный коллектив учёных во главе с гарв...,Гарвардские ученые начали проект Galileo для п...,"Группа ученых из известных университетов, возг...",Международная команда ученых под руководством ...,"Международный коллектив ученых, включая Ави Лё...",Международная группа ученых под руководством А...


# Yandex GPT

In [ ]:
folder_id = 'xxx'
api_key = 'xxx'

>**Системный промпт**

In [ ]:
system_prompt = 'Перескажи статью в 25 - 50 слов'

>**Пользовательский промпт**

In [ ]:
user_prompt = f'Перескажи статью.\nСтатья : '

In [ ]:
class GiveMeResults():
    def __init__(self, model_name = 'yandexgpt',    # llama yandexgpt
                       model_version = 'rc',        # latest  rc
                       folder_id = folder_id,
                       auth = api_key):

        sdk = YCloudML(folder_id = folder_id, auth = auth)
        self.model = sdk.models.completions(model_name, model_version = model_version)
        self.model = self.model.configure(temperature=0.3)

    def get_answers(self, texts, system_prompt = system_prompt, user_prompt = user_prompt):
        self.results = []
        for text in tqdm(texts):
            answer = self.model.run([{"role" : "system", "text" : system_prompt},
                                {"role": "user", "text" : user_prompt + text}])
            answer = answer.alternatives[0].text
            self.results.append(answer)

In [ ]:
texts = list(df.text.values)

>**YandexGPT5_PRO_RC**

In [ ]:
classifier = GiveMeResults()
classifier.get_answers(texts)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
#Заменяем это сообщение на None
df['YandexGPT5_PRO_RC'] = classifier.results
block_yandex = 'В интернете есть много сайтов с информацией на эту тему. [Посмотрите, что нашлось в поиске](https://ya.ru)'
block_idx = df['YandexGPT5_PRO_RC'].apply(lambda x: x==block_yandex)
df.loc[block_idx, 'YandexGPT5_PRO_RC'] = None

>**YandexGPT5_LITE_RC**

In [ ]:
classifier = GiveMeResults(model_name = 'yandexgpt-lite')
classifier.get_answers(texts)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
#Заменяем это сообщение на None
df['YandexGPT5_Lite_RC'] = classifier.results
block_yandex = 'В интернете есть много сайтов с информацией на эту тему. [Посмотрите, что нашлось в поиске](https://ya.ru)'
block_idx = df['YandexGPT5_Lite_RC'].apply(lambda x: x==block_yandex)
df.loc[block_idx, 'YandexGPT5_Lite_RC'] = None

In [ ]:
df.to_excel(path + "Рефераты_моделей.xlsx", index=False)

# GigaChat

In [ ]:
prompt = dict()
prompt['system'] = "Перескажи новостную статью в 25 - 50 слов"
prompt['user'] = f"Статья:\n"

In [ ]:
class AskGigaChat():
    def __init__(self, prompt = prompt,
                 model_name = "GigaChat-2"):#GigaChat
        api_key = 'xxx'

        self.prompt = prompt
        self.model = GigaChat(credentials = api_key,
                              verify_ssl_certs=False,
                              model=model_name)

    def ask(self, text):
        messages = [SystemMessage(content = self.prompt['system']),
                    HumanMessage(content = self.prompt['user'] + text)]

        parser = StrOutputParser()
        result = self.model.invoke(messages)
        answer = parser.invoke(result)
        return answer

    def get_answers(self, texts):
        self.results = []
        for text in tqdm(texts):
            self.results.append(self.ask(text))

In [2]:
giga = AskGigaChat()
giga.get_answers(df.text.values)

In [ ]:
df['GigaChat_2_Lite'] = giga.results

In [ ]:
block_giga1 = 'Как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.'
block_giga2 = 'Не люблю менять тему разговора, но вот сейчас тот самый случай.'
block_giga3 = 'Что-то в вашем вопросе меня смущает. Может, поговорим на другую тему?'
blocks_giga = [block_giga1, block_giga2, block_giga3]
block_idx = df['GigaChat_2_Lite'].apply(lambda x: x in blocks_giga)
df.loc[block_idx, 'GigaChat_2_Lite'] = None

In [ ]:
df.to_excel(path + "Рефераты_моделей.xlsx", index=False)

In [ ]:
df[df['GigaChat_2_Lite'].notna()].shape

(188, 7)

# Grok

In [ ]:
keys = ['xxx', #Вова
        'xxx'] #Рем
clients = [Groq(api_key = api_key) for api_key in keys]

In [ ]:
models = ["llama-3.3-70b-versatile", "mixtral-8x7b-32768",
        'deepseek-r1-distill-llama-70b', 'meta-llama/llama-4-scout-17b-16e-instruct']
models = ["llama-3.3-70b-versatile", 'deepseek-r1-distill-llama-70b', 'meta-llama/llama-4-scout-17b-16e-instruct']


In [ ]:
model_params = {
  'llama-3.3-70b-versatile': {'temperature' : 0.5,
                              'max_completion_tokens' : 1000},

  'deepseek-r1-distill-llama-70b': {'temperature' : 0.6,
                                    'max_completion_tokens' : 5000},

  'meta-llama/llama-4-scout-17b-16e-instruct': {'temperature' : 0.8,
                                        'max_completion_tokens' : 1000},
}

model_params['mixtral-8x7b-32768'] = model_params['llama-3.3-70b-versatile']

In [ ]:
prompt = dict()
prompt['system'] = "Перескажи статью в 25 - 50 слов"
prompt['system'] = "Перескажи статью в пределах 2 - 4 предложений"
prompt['system'] = "Перескажи статью около 50 слов "
prompt['system'] = 'Summarize the article in approximately 45 words, using 3 to 4 full sentences. Focus on the main ideas without omitting key details.'
prompt['system'] = 'You are a professional summarizer who specializes in summarizing Russian-language texts. You always write in natural, fluent Russian, using 3–4 full sentences and about 45 words.'
prompt['system'] = 'You are a professional assistant specializing in summarizing Russian-language texts. Your summaries should always be written in natural, fluent Russian, using 3–4 complete sentences and approximately 45 words. Focus on preserving the core ideas without excessive shortening or omission. Always write the summary in Russian.'


prompt['user'] = f"Перескажи статью.\nСтатья:"
prompt['user'] = 'Summarize the following article in about 45 words using 3–4 complete sentences.\n'
prompt['user'] = 'Перескажи следующий текст на русском языке в виде 3–4 полных предложений, объёмом примерно 45 слов. Сделай пересказ кратким, но содержательным.\nСтатья:'



In [ ]:
class Ask_Groq():
    def __init__(self, clients = clients,
                 model = 'llama-3.3-70b-versatile',
                 model_params = model_params, prompt = prompt):

        self.clients = clients
        self.current_client = 0
        self.model = model
        self.model_params = model_params
        self.prompt = prompt
        self.client_history = []
        '''
        self.model_answers - хранение ответов одной модели
        '''
    def available_client(self):
        #функция меняет агента каждый вызов.
        self.current_client = (self.current_client + 1) % len(self.clients)
        client = self.clients[self.current_client]
        self.client_history.append(self.current_client)
        return client

    def ask_model(self, model, text, prompt = prompt):
        time.sleep(1)#Задержка в 1 секунду
        client = self.available_client()
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": prompt['system']
                },
                {
                    "role": "user",
                    "content": prompt['user'] + f"\nТекст: {text}"
                },
            ],

            temperature = self.model_params[model]['temperature'],
            max_completion_tokens = self.model_params[model]['max_completion_tokens'],
            top_p=1,
            stream=False
        )

        return completion.choices[0].message.content


    def get_answers(self, texts):
        self.results = []
        for text in tqdm(texts, desc = f'Работает {self.model}'):
            self.results.append(self.ask_model(self.model, text))


## Llama 4

In [ ]:
llama4 = Ask_Groq(model = 'meta-llama/llama-4-scout-17b-16e-instruct')
llama4.get_answers(df.text.values)

In [ ]:
df['meta-llama/llama-4-scout-17b-16e-instruct'] = llama4.results
df.to_excel(path + "Рефераты_моделей.xlsx", index=False)

## Deepseek

In [ ]:
model_name = 'deepseek-r1-distill-llama-70b'
deepseek = Ask_Groq(model = model_name)
deepseek.get_answers(df.text.values)

Работает deepseek-r1-distill-llama-70b:   0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
df.head(2)

,text,summary,rubert_ext_sum,rut5_abs_sum,gpt_abs_sum,YandexGPT5_PRO_RC,GigaChat_2_Lite,meta-llama/llama-4-scout-17b-16e-instruct,deepseek-r1-distill-llama-70b
0,Американские вооруженные силы должны быть гото...,Экс-президент США продолжает критиковать админ...,Американские вооруженные силы должны быть гото...,"Бывший глава Белого дома Дональд Трамп заявил,...","Бывший президент США Дональд Трамп заявил, что...",NaN,NaN,"Дональд Трамп заявил, что американские вооруже...","Дональд Трамп заявил, что США должны быть гото..."
1,"Международный коллектив ученых из Гарварда , П...",Международный коллектив под руководством гарва...,"Международный коллектив ученых из Гарварда , П...","Международный коллектив ученых из Гарварда, Пр...",Ученые из Гарварда и Принстона объявили о нача...,Международный коллектив учёных во главе с гарв...,Гарвардские ученые начали проект Galileo для п...,"Группа ученых из известных университетов, возг...",Международная команда ученых под руководством ...


In [ ]:
df[model_name] = deepseek.results
df.iloc[:, -1] = df.iloc[:, -1].str.split('</think>').apply(lambda x:x[1]).str.strip()
df.to_excel(path + "Рефераты_моделей.xlsx", index=False)